# Description:

- Create CSV and accompanying plots of appointments for a given scenario

- First of all calculate appointments required by each staff type & appointment mode
{delivered, missed, unmet demand}

- Total demand = Sum{delivered, missed, unmet demand}

- FTE Calculations are in the repo.

 

Per month

Per alliance

Exam question/Objective:
Need the appointments for each of the healthcare staff mentioned.

For each demand scenario

For each capacity scenario

As we will be doing multiple ‘runs’ per scenario we will need mean, median columns. Consider other useful metrics to report confidence.

Key Data Sources:
Output from simulation

This is a very large dictionary

Consider writing a function to break it down.

Methodology/Approach:
Using the met demand + unmet demand, determine how much FTE is required.

Subtasks:
Wrangle data from simulation output

Outputs:
Github branch

CSV

forecast yaml file, run for all regression models

## Library Imports

In [2]:
import os
from pathlib import Path
if 'notebooks' in str(Path.cwd()):
    os.chdir('..')

# Library imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
from matplotlib.ticker import FuncFormatter

# project imports from src
from src.schemas import DataCatalog
from src.various_methods import PlotCounter

# Importing SNEE styles
from sneeifstyles import mpl_style
mpl_style()

In [10]:
## Constants
SNEE_SUB_ICB = ['06L','06T','07K']
NOTEBOOK_ALIAS = "Appointments"
GP_APPOINTMENTS_CATALOG_NAME:str = 'Appointments in General Practice, August 2024'
GP_WORKFORCE_CATALOG:str = 'General Practice workforce'

# Loading the Data Catalog
catalog =  DataCatalog.load_from_yaml("data_catalog.yaml")

# Initializing the plotCounter object
plot_counter = PlotCounter(name=NOTEBOOK_ALIAS)

# set up output directories
for i in ['outputs/assumptions', 'outputs/plots', 'outputs/tables']:
    if not os.path.exists(i):
        os.makedirs(i)     

In [12]:
workforce_entry =  catalog.get_catalog_entry_by_name(GP_WORKFORCE_CATALOG)
workforce_df = workforce_entry.load()

appointments_catalog_entry = catalog.get_catalog_entry_by_name(GP_APPOINTMENTS_CATALOG_NAME)
appointments_df = appointments_catalog_entry.load()

In [8]:
workforce_df.head()

,YEAR,Month,COMM_REGION_CODE,COMM_REGION_NAME,ICB_CODE,ICB_NAME,SUB_ICB_CODE,SUB_ICB_NAME,DATA_SOURCE,UNIQUE_IDENTIFIER,STAFF_GROUP,DETAILED_STAFF_ROLE,STAFF_ROLE,COUNTRY_QUALIFICATION_AREA,COUNTRY_QUALIFICATION_GROUP,AGE_BAND,AGE_YEARS,GENDER,FTE
0,2024,8,Y60,Midlands,QJ2,NHS Derby and Derbyshire ICB,15M,NHS Derby and Derbyshire ICB - 15M,Provided,103041,GP,Salaried By Practice,Salaried GPs,UK,UK,30-34,30.0,Male,0.640000
1,2024,8,Y60,Midlands,QJ2,NHS Derby and Derbyshire ICB,15M,NHS Derby and Derbyshire ICB - 15M,Provided,172433,Direct Patient Care,Healthcare Assistant,Healthcare Assistants,Not Applicable,Not Applicable,55-59,55.0,Female,0.906667
2,2024,8,Y60,Midlands,QJ2,NHS Derby and Derbyshire ICB,15M,NHS Derby and Derbyshire ICB - 15M,Provided,179320,Admin/Non-Clinical,Receptionist,Receptionists,Not Applicable,Not Applicable,Under 25,23.0,Female,0.800000
3,2024,8,Y60,Midlands,QJ2,NHS Derby and Derbyshire ICB,15M,NHS Derby and Derbyshire ICB - 15M,Provided,130840,Admin/Non-Clinical,Receptionist,Receptionists,Not Applicable,Not Applicable,55-59,58.0,Female,0.520000
4,2024,8,Y60,Midlands,QJ2,NHS Derby and Derbyshire ICB,15M,NHS Derby and Derbyshire ICB - 15M,Provided,25423,Nurses,Practice Nurse,Practice Nurses,Not Applicable,Not Applicable,40-44,41.0,Female,0.480000


In [13]:
appointments_df

,SUB_ICB_LOCATION_CODE,SUB_ICB_LOCATION_ONS_CODE,SUB_ICB_LOCATION_NAME,ICB_ONS_CODE,REGION_ONS_CODE,Appointment_Date,ACTUAL_DURATION,COUNT_OF_APPOINTMENTS
0,00L,E38000130,NHS North East and North Cumbria ICB - 00L,E54000050,E40000012,01MAR2022,11-15 Minutes,1281
1,00L,E38000130,NHS North East and North Cumbria ICB - 00L,E54000050,E40000012,01MAR2022,16-20 Minutes,873
2,00L,E38000130,NHS North East and North Cumbria ICB - 00L,E54000050,E40000012,01MAR2022,1-5 Minutes,1359
3,00L,E38000130,NHS North East and North Cumbria ICB - 00L,E54000050,E40000012,01MAR2022,6-10 Minutes,1761
4,00L,E38000130,NHS North East and North Cumbria ICB - 00L,E54000050,E40000012,01MAR2022,31-60 Minutes,440
...,...,...,...,...,...,...,...,...
595540,X2C4Y,E38000254,NHS West Yorkshire ICB - X2C4Y,E54000054,E40000012,31AUG2024,31-60 Minutes,7
595541,X2C4Y,E38000254,NHS West Yorkshire ICB - X2C4Y,E54000054,E40000012,31AUG2024,21-30 Minutes,16
595542,X2C4Y,E38000254,NHS West Yorkshire ICB - X2C4Y,E54000054,E40000012,31AUG2024,16-20 Minutes,30
595543,X2C4Y,E38000254,NHS West Yorkshire ICB - X2C4Y,E54000054,E40000012,31AUG2024,11-15 Minutes,28
